In [7]:
import sys
from lxml import html as htmlRenderer
import requests
import json
from datetime import date
from datetime import datetime
from random import *
import base64

In [18]:
# urlInfoComments = "https://grupogodo1.bootstrap.fyre.co/api/v1.1/public/comments/ncomments/{}.json"
urlInfoComments = "https://data.livefyre.com/bs3/v3.1/grupogodo1.fyre.co/{}/{}/init"
urlGetComments = "https://data.livefyre.com/bs3/v3.1/grupogodo1.fyre.co/{}/{}/{}.json"
urlXpathQuery = "//div[@class='main']//i/@data-href"

In [9]:
def extractComments(commentsList=None, authorObjectList=None, urlNoticia=None, specialCase=None):
    print(" \t -> parsing comments list with -{}- elements:".format(len(commentsList)))
    parsedComments = []
    listToParse = commentsList[1] if specialCase else commentsList
    for commentObj in listToParse:
        contentObj = commentObj.get("content", None)
        isValidContent = "bodyHtml" in contentObj.keys()
        if isValidContent:
            fechaObj = datetime.fromtimestamp(contentObj["createdAt"])
            fechaStr = fechaObj.strftime("%d/%m/%Y-%H:%M:%S")
            fechaArr = fechaStr.split("-")
            parsedComment = {
                "urlNoticia": urlNoticia,
                "fecha": fechaArr[0],
                "hora": fechaArr[1],
                "user": authorObjectList[contentObj['authorId']]['displayName'],
                "commentario": contentObj['bodyHtml']
            }
            parsedComments.append(parsedComment)
    return parsedComments


In [25]:
def lookupForComments(renderedPageHtml=None, url=None):
    commendInfoKeysElArr = renderedPageHtml.xpath("//label[@class='livefyre-commentcount']")
    pageComments = []
    if len(commendInfoKeysElArr) > 0:
        commentInfoEl = commendInfoKeysElArr[0]
        commentSiteId = commentInfoEl.get("data-lf-site-id")
        commentArticleId = commentInfoEl.get("data-lf-article-id")
        if (commentSiteId != "" and commentArticleId != ""):
            print(" \t-> data-lf-site-id to get comments is: {}".format(commentSiteId))
            print(" \t-> data-lf-article-id to get comments is: {}".format(commentArticleId))

            # Get Comments Info
            urlParam = "{}:{}".format(commentSiteId, commentArticleId)
            urlParamBase64 = str(base64.b64encode(urlParam.encode("utf-8")), "utf-8")
            commentElArticleIdEncoded = str(base64.b64encode(commentArticleId.encode("utf-8")), "utf-8")
            urlInfoCommentsEncoded = urlInfoComments.format(commentSiteId, commentElArticleIdEncoded)
            print(" \t-> getting information for article: {}".format(urlInfoCommentsEncoded))
            responseInfoComments = requests.get(urlInfoCommentsEncoded)
            infoComments = json.loads(responseInfoComments.text)
            if infoComments["collectionSettings"]["numVisible"] > 0:
                # La info dice que hay comentarios, se procede a obtenerlos
                pages = infoComments["collectionSettings"]["archiveInfo"]["nPages"]
                print(" \t-> total of comments for current article: {}".format(
                    infoComments["collectionSettings"]["numVisible"]))
                print(" \t-> total of pages for current article: {}".format(pages))
                
                for page in range(pages):
                    urlGetCommentsEncoded = urlGetComments.format(commentSiteId, commentElArticleIdEncoded, page)
                    print(" \t -> getting comments with url: {}".format(urlGetCommentsEncoded))
                    responseComments = requests.get(urlGetCommentsEncoded)
                    commentsResponse = json.loads(responseComments.text)
                    pageComments = pageComments + extractComments(commentsResponse['content'],
                                                        commentsResponse['authors'], url)
                print(" \t -> retrieved total of {} comments".format(len(pageComments)))
                print(
                    "#############################################################################################")
    return pageComments

In [35]:
url="https://www.lavanguardia.com/local/barcelona/20190514/462232803499/josep-bou-pp-elecciones-municipales-barcelona.html"
renderUrl = requests.get(url)
renderedPage = htmlRenderer.fromstring(renderUrl.text)
comments = lookupForComments(renderedPage, url)

 	-> data-lf-site-id to get comments is: 351112
 	-> data-lf-article-id to get comments is: 9eb06270-75d2-11e9-b72a-308b3042036b
 	-> getting information for article: https://data.livefyre.com/bs3/v3.1/grupogodo1.fyre.co/351112/OWViMDYyNzAtNzVkMi0xMWU5LWI3MmEtMzA4YjMwNDIwMzZi/init
 	-> total of comments for current article: 8
 	-> total of pages for current article: 1
 	 -> getting comments with url: https://data.livefyre.com/bs3/v3.1/grupogodo1.fyre.co/351112/OWViMDYyNzAtNzVkMi0xMWU5LWI3MmEtMzA4YjMwNDIwMzZi/0.json
 	 -> parsing comments list with -17- elements:
 	 -> retrieved total of 8 comments
#############################################################################################


In [32]:
len(comments)

678